In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
import os
from tensorflow_model_optimization.quantization.keras import vitis_quantize


shape_range =range(75,491,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,3])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0

# create the base pre-trained model
    base_model = InceptionV3(input_shape=(shape_x,shape_y,3),weights=None, include_top=False)

    # add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    # let's add a fully-connected layer
    x = Dense(1024, activation='relu')(x)
    # and a logistic layer -- let's say we have 200 classes
    predictions = Dense(10, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in base_model.layers:
        layer.trainable = False

    # compile the model (should be done *after* setting layers to non-trainable)
    #model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    model.summary()
    # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    # history = model.fit(X_train, Y_train, epochs=1)
    # model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
    # quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
    --model ./quantized.h5 \
    --arch ./arch.json \
    --output_dir ./xmodel_Inception_v3 \
    --net_name deploy
    os.rename("./xmodel_Inception_v3/deploy.xmodel","./xmodel_Inception_v3/Inception_v3_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))



Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 75, 75, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, 37, 37, 32)   864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_376 (BatchN (None, 37, 37, 32)   96          conv2d_376[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, 37, 37, 32)   0           batch_normalization_376[0][0]    
____________________________________________________________________________________________

[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 19s 2s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...


In [2]:
from keras.models import Model
from keras.layers import Activation, Input, concatenate, Dropout, Dense, Flatten
from keras.layers import MaxPooling2D, Conv2D, AveragePooling2D, BatchNormalization
import numpy as np
import os
from tensorflow_model_optimization.quantization.keras import vitis_quantize

def create(image_height, image_width,classes_num=10,image_channel=1):
    inputs = Input((image_height, image_width, image_channel))


    net = block_stem(inputs)

    # 4 x Inception-A ( Output: 35 x 35 x 384 )
    for i in range(4):
        net = block_inception_a(net)

    # Reduction-A ( Output: 17 x 17 x 1024 )
    net = block_reduction_a(net)

    # 7 x Inception-B ( Output: 17 x 17 x 1024 )
    for i in range(7):
        net = block_inception_b(net)

    # Reduction-B ( Output: 8 x 8 x 1536 )
    net = block_reduction_b(net)

    # 3 x Inception-C ( Output: 8 x 8 x 1536 )
    for i in range(3):
        net = block_inception_c(net)

    # Average Pooling ( Output: 1536 )
    net = AveragePooling2D((8, 8))(net)

    # Dropout ( keep 0.8 )
    net = Dropout(0.2)(net)
    net = Flatten()(net)

    # Output
    outputs = Dense(units=classes_num, activation='softmax')(net)

    return Model(inputs, outputs, name='Inception-v4')


def block_stem(inputs):
    net = conv2d(inputs, 32, (3, 3), strides=(2, 2), padding='valid')
    net = conv2d(net, 32, (3, 3), padding='valid')
    net = conv2d(net, 64, (3, 3))

    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(net)
    branch_2 = conv2d(net, 96, (3, 3), strides=(2, 2), padding='valid')

    net = concatenate([branch_1, branch_2])

    branch_1 = conv2d(net, 64, (1, 1))
    branch_1 = conv2d(branch_1, 96, (3, 3), padding='same')

    branch_2 = conv2d(net, 64, (1, 1))
    branch_2 = conv2d(branch_2, 64, (7, 1))
    branch_2 = conv2d(branch_2, 64, (1, 7))
    branch_2 = conv2d(branch_2, 96, (3, 3), padding='same')

    net = concatenate([branch_1, branch_2])

    branch_1 = conv2d(net, 192, (3, 3), strides=(2, 2), padding='same')  # 与论文不同
    branch_2 = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(net)

    net = concatenate([branch_1, branch_2])

    return net


def block_inception_a(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 96, (1, 1))

    branch_2 = conv2d(inputs, 96, (1, 1))

    branch_3 = conv2d(inputs, 64, (1, 1))
    branch_3 = conv2d(branch_3, 96, (3, 3))

    branch_4 = conv2d(inputs, 64, (1, 1))
    branch_4 = conv2d(branch_4, 96, (3, 3))
    branch_4 = conv2d(branch_4, 96, (3, 3))

    return concatenate([branch_1, branch_2, branch_3, branch_4])


def block_inception_b(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 128, (1, 1))

    branch_2 = conv2d(inputs, 384, (1, 1))

    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (1, 7))
    branch_3 = conv2d(branch_3, 256, (7, 1))  # 与论文不同

    branch_4 = conv2d(inputs, 192, (1, 1))
    branch_4 = conv2d(branch_4, 192, (1, 7))
    branch_4 = conv2d(branch_4, 224, (7, 1))
    branch_4 = conv2d(branch_4, 224, (1, 7))
    branch_4 = conv2d(branch_4, 256, (7, 1))

    return concatenate([branch_1, branch_2, branch_3, branch_4])


def block_inception_c(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(inputs)
    branch_1 = conv2d(branch_1, 256, (1, 1))

    branch_2 = conv2d(inputs, 256, (1, 1))

    branch_3 = conv2d(inputs, 384, (1, 1))
    branch_3_1 = conv2d(branch_3, 256, (1, 3))
    branch_3_2 = conv2d(branch_3, 256, (3, 1))

    branch_4 = conv2d(inputs, 384, (1, 1))
    branch_4 = conv2d(branch_4, 448, (1, 3))
    branch_4 = conv2d(branch_4, 512, (3, 1))
    branch_4_1 = conv2d(branch_4, 256, (3, 1))
    branch_4_2 = conv2d(branch_4, 256, (1, 3))

    return concatenate([branch_1, branch_2, branch_3_1, branch_3_2, branch_4_1, branch_4_2])


def block_reduction_a(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    branch_2 = conv2d(inputs, 384, (3, 3), strides=(2, 2), padding='valid')

    branch_3 = conv2d(inputs, 192, (1, 1))
    branch_3 = conv2d(branch_3, 224, (3, 3))
    branch_3 = conv2d(branch_3, 256, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([branch_1, branch_2, branch_3])


def block_reduction_b(inputs):
    branch_1 = MaxPooling2D((3, 3), strides=(2, 2), padding='valid')(inputs)

    branch_2 = conv2d(inputs, 192, (1, 1))
    branch_2 = conv2d(branch_2, 192, (3, 3), strides=(2, 2), padding='valid')

    branch_3 = conv2d(inputs, 256, (1, 1))
    branch_3 = conv2d(branch_3, 256, (1, 7))
    branch_3 = conv2d(branch_3, 320, (7, 1))
    branch_3 = conv2d(branch_3, 320, (3, 3), strides=(2, 2), padding='valid')

    return concatenate([branch_1, branch_2, branch_3])


def conv2d(net, filters, kernel_size, strides=(1, 1), padding='same'):
    net = Conv2D(filters=filters, kernel_size=kernel_size, strides=strides, padding=padding, use_bias=False)(net)
    net = BatchNormalization()(net)
    net = Activation('relu')(net)
    return net

shape_range =range(299,491,32)
for shape_x in shape_range:
    shape_y = shape_x
    X_train=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_train=np.random.randint(0,10,size=[60,])
    X_test=np.random.randint(0,255,size=[60,shape_x,shape_y,1])
    Y_test=np.random.randint(0,10,size=[60,])
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0

    model = create(shape_x, shape_y)
    model.summary()
  

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, Y_train, epochs=1)
    model.get_weights()[0].dtype
    quantizer = vitis_quantize.VitisQuantizer(model)
    quantized_model = quantizer.quantize_model(calib_dataset = X_test[1:8], weight_bit=8, activation_bit=8)
    quantized_model.compile(loss='sparse_categorical_crossentropy', metrics=["accuracy"])
    quantized_model.save('quantized.h5')
    !vai_c_tensorflow2 \
    --model ./quantized.h5 \
    --arch ./arch.json \
    --output_dir ./xmodel_Inception_v4 \
    --net_name deploy
    os.rename("./xmodel_Inception_v4/deploy.xmodel","./xmodel_Inception_v4/Inception_v4_{shape_x}_{shape_y}.xmodel".format(shape_x=shape_x,shape_y=shape_y))


Model: "Inception-v4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 1) 0                                            
__________________________________________________________________________________________________
conv2d_298 (Conv2D)             (None, 149, 149, 32) 288         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_298 (BatchN (None, 149, 149, 32) 128         conv2d_298[0][0]                 
__________________________________________________________________________________________________
activation_298 (Activation)     (None, 149, 149, 32) 0           batch_normalization_298[0][0]    
_______________________________________________________________________________________

2/2 [==============================] - 31s 11s/step - loss: 2.4020 - accuracy: 0.1000
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 39s 4s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 60s 60s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras vers

2/2 [==============================] - 36s 13s/step - loss: 2.7262 - accuracy: 0.1167
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 42s 5s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 61s 61s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras vers

2/2 [==============================] - 41s 16s/step - loss: 2.7568 - accuracy: 0.1500
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 43s 5s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 62s 62s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras vers

2/2 [==============================] - 47s 18s/step - loss: 2.6978 - accuracy: 0.1667
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 38s 4s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 63s 63s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras vers

2/2 [==============================] - 57s 22s/step - loss: 2.8272 - accuracy: 0.1000
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 38s 4s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 65s 65s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras vers

2/2 [==============================] - 64s 25s/step - loss: 2.6426 - accuracy: 0.1333
[VAI INFO] Update custom_layer_type: []
[VAI INFO] Update activation_bit: 8
[VAI INFO] Update weight_bit: 8
[VAI INFO] Start CrossLayerEqualization...
10/10 [==============================] - 38s 4s/step
[VAI INFO] CrossLayerEqualization Done.
[VAI INFO] Start Quantize Calibration...
1/1 [==============================] - 65s 65s/step
[VAI INFO] Quantize Calibration Done.
[VAI INFO] Start Post-Quantize Adjustment...
[VAI INFO] Post-Quantize Adjustment Done.
[VAI INFO] Quantization Finished.
**************************************************
* VITIS_AI Compilation - Xilinx Inc.
**************************************************
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['./quantized.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/deploy_org.xmodel', proto=None)
[INFO] tensorflow2 model: /workspace/DPU-PYNQ/host/quantized.h5
[INFO] keras vers